In [4]:
# ===========================================
# Notebook: 01_EDA_ETH.ipynb
# Test działania modułu data_loader
# ===========================================

import os
import sys
import pandas as pd

# --- Ustawiamy ścieżkę do katalogu projektu ---
project_root = os.path.abspath(os.path.join(".."))  # jeden poziom wyżej niż notebooks/
if project_root not in sys.path:
    sys.path.append(project_root)

# --- Import funkcji z modułu data_loader ---
from src.data_loader import load_train_test_data

# --- Uruchamiamy funkcję, która wczyta dane i policzy log-returny ---
train_df, test_df = load_train_test_data()

# --- Sprawdzamy podstawowe informacje ---
print("\n--- TRAIN ---")
train_df.info()
print(train_df.head())

print("\n--- TEST ---")
test_df.info()
print(test_df.head())

# --- Szybka kontrola log_return ---
print("\nZakres log_return (train):", train_df["log_return"].min(), "→", train_df["log_return"].max())
print("Zakres log_return (test):", test_df["log_return"].min(), "→", test_df["log_return"].max())


✅ Dane załadowane poprawnie.
Train shape: (17279, 11), Test shape: (8927, 11)
Train range: 2024-01-01 00:05:00 → 2024-02-29 23:55:00
Test range:  2024-03-01 00:05:00 → 2024-03-31 23:55:00

--- TRAIN ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17279 entries, 0 to 17278
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   open_time           17279 non-null  datetime64[ns]
 1   open                17279 non-null  float64       
 2   high                17279 non-null  float64       
 3   low                 17279 non-null  float64       
 4   close               17279 non-null  float64       
 5   volume              17279 non-null  float64       
 6   quote_asset_volume  17279 non-null  float64       
 7   num_trades          17279 non-null  int64         
 8   taker_buy_base      17279 non-null  float64       
 9   taker_buy_quote     17279 non-null  float64       
 10  log_return 